In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [13]:
import asyncio
from lib.zebra_grid import get_zebra_grid_tasks
from lib.tasks import ChatCompletionParams, get_task_results
import openai
import os

zebra_grid_tasks = list(get_zebra_grid_tasks())
benchmark_tasks = zebra_grid_tasks[:80]
val_tasks = zebra_grid_tasks[:64]
test_tasks = zebra_grid_tasks[64:128]
train_tasks = zebra_grid_tasks[128:]
len(benchmark_tasks), len(val_tasks), len(test_tasks), len(train_tasks)

(80, 64, 64, 872)

In [17]:
fireworks = openai.AsyncOpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY"),
)
openrouter = openai.AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1", api_key=os.getenv("OPENROUTER_API_KEY")
)
together = openai.AsyncOpenAI(
    base_url="https://api.together.xyz/v1", api_key=os.getenv("TOGETHER_API_KEY")
)

results = await asyncio.gather(
    get_task_results(
        tasks=benchmark_tasks,
        client=fireworks,
        model="accounts/fireworks/models/deepseek-r1",
        params=ChatCompletionParams(
            max_tokens=2**17,
            logprobs=True,
            top_logprobs=5,
        ),
        pbar_desc="deepseek-r1",
        prices=(8.0, 8.0),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="deepseek/deepseek-r1-distill-qwen-1.5b",
        pbar_desc="r1-qwen-1.5b",
        prices=(0.18, 0.18),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="qwen/qwen-2.5-7b-instruct",
        pbar_desc="qwen-2.5-7b",
        params=ChatCompletionParams(
            extra_body={"provider": {"order": ["DeepInfra"], "allow_fallbacks": False}},
        ),
        prices=(0.0025, 0.005),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="deepseek/deepseek-r1-distill-qwen-14b",
        pbar_desc="r1-qwen-14b",
        prices=(1.6, 1.6),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="deepseek/deepseek-r1-distill-qwen-32b",
        params=ChatCompletionParams(
            extra_body={"provider": {"order": ["DeepInfra"], "allow_fallbacks": False}},
        ),
        pbar_desc="r1-qwen-32b",
        prices=(0.12, 0.18),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="deepseek/deepseek-r1-distill-llama-70b:free",
        params=ChatCompletionParams(
            extra_body={"provider": {"order": ["Targon"], "allow_fallbacks": False}},
        ),
        pbar_desc="r1-llama-70b:targon",
        prices=(0.0, 0.0),
    ),
    get_task_results(
        tasks=benchmark_tasks,
        client=openrouter,
        model="deepseek/deepseek-r1-distill-llama-70b",
        params=ChatCompletionParams(
            extra_body={"provider": {"order": ["SambaNova"], "allow_fallbacks": False}},
        ),
        pbar_desc="r1-llama-70b:samba",
        prices=(0.7, 1.4),
    ),
)

deepseek-r1:   0%|          | 0/80 [00:00<?, ?it/s]

r1-qwen-1.5b:   0%|          | 0/80 [00:00<?, ?it/s]

qwen-2.5-7b:   0%|          | 0/80 [00:00<?, ?it/s]

r1-qwen-14b:   0%|          | 0/80 [00:00<?, ?it/s]

r1-qwen-32b:   0%|          | 0/80 [00:00<?, ?it/s]

r1-llama-70b:targon:   0%|          | 0/80 [00:00<?, ?it/s]

r1-llama-70b:samba:   0%|          | 0/80 [00:00<?, ?it/s]

In [10]:
raise [exception for result in results[1][0] for exception in result.exceptions][0]

AssertionError: 